# Azure ML Managed Endpoint

In [ ]:
import os

from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential

from azure.ai.ml.constants import BatchDeploymentOutputAction, AssetTypes
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, BatchRetrySettings, ModelBatchDeployment, CodeConfiguration, ModelBatchDeploymentSettings, Data
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment, Model, Environment

from dotenv import load_dotenv
load_dotenv(override=True)

<h5>Azure ML Client の設定</h5>
環境変数からAzureの認証情報を取得し、MLClientを初期化します。

In [ ]:
subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP")
workspace = os.getenv("AML_WORKSPACE_NAME")
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

### Batch Endpoint
👉
##### [バッチ エンドポイントにスコアリング用のモデルをデプロイする](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-batch-model-deployments?view=azureml-api-2&tabs=python)

<h5>バッチエンドポイントの作成</h5>
大量データの一括推論用のバッチエンドポイントを作成します。

In [ ]:
# create a batch endpoint
endpoint = BatchEndpoint(
    name="ep-batch-oh4ml-001",
    description="this is a sample batch endpoint",
    tags={"type": "test"},
)

ml_client.begin_create_or_update(endpoint)

### Deployment

<h5>バッチデプロイメントの作成</h5>
登録済みモデルを使用してバッチ処理用のデプロイメントを作成します。

In [ ]:
deployment = ModelBatchDeployment(
    name="batch-deployment-01",
    description="batch deployment test",
    endpoint_name="ep-batch-oh4ml-001",
    model="diabetes_model_01@latest",
    code_configuration=CodeConfiguration(
        code="./scripts", scoring_script="score-diabetes.py"
    ),
    environment="diabetes-env-01@latest",
    compute="demo-cpucluster1",
    settings=ModelBatchDeploymentSettings(
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        instance_count=2,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
        logging_level="info",
    ),
)

ml_client.begin_create_or_update(deployment).result()

<h5> デフォルトデプロイメントの設定</h5>
作成したデプロイメントをエンドポイントのデフォルトとして設定します。

In [ ]:
endpoint = ml_client.batch_endpoints.get("ep-batch-oh4ml-001")
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

##### Upload Data for test

<h5> テスト用データの登録</h5>
バッチ推論で使用するテスト用データセットを登録します。

In [ ]:
data_uri = "./diabetes_query.csv"

data = Data(
    path = data_uri,
#    type = AssetTypes.URI_FOLDER,
    type = AssetTypes.URI_FILE,
    description = "test for demo",
    name = "diabetes_query_01",
    version = '1'
)

ml_client.data.create_or_update(data)

### Invoke

<h5> バッチエンドポイントの実行</h5>
登録したテストデータを使用してバッチ推論ジョブを実行します。

In [ ]:
input = Input(
    type = AssetTypes.URI_FILE,
    #path = "azureml://datastores/workspaceblobstore/paths/diabetes/diabetes_query.csv"
    path = "diabetes_query_01@latest" #/versions/2"
)

# invoke the endpoint for batch scoring job
job = ml_client.batch_endpoints.invoke(
    endpoint_name = "ep-batch-oh4ml-001",
    input = input,
    #deployment_name = "batch-deployment-01",  # name is not required as default deployment is set
    params_override = [{"mini_batch_size": "5"}, {"compute.instance_count": "2"}],
)

###  Get the details of the invoked job
Let us get details and logs of the invoked job

<h5> ジョブ実行状況の確認</h5>
実行中のバッチジョブの状況とログを確認します。

In [ ]:
# get the details of the job
job_name = job.name
batch_job = ml_client.jobs.get(name=job_name)
print(batch_job.status)
# stream the job logs
ml_client.jobs.stream(name=job_name)

### Online Endpoint
👉
##### [オンライン エンドポイントを使用して機械学習モデルをデプロイおよびスコア付けする](https://learn.microsoft.com/ja-jp/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=python)

### Create Endpoint

<h5>オンラインエンドポイントの作成</h5>
リアルタイム推論用のオンラインエンドポイントを作成します。

In [ ]:
online_endpoint_name = "ep-online-oh4ml-01"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="endpoint online oh4ml 01",
    auth_mode="key",
    tags={"version": "001"},
)

endpoint = ml_client.begin_create_or_update(endpoint)


<h5>エンドポイント状態の確認</h5>
作成したオンラインエンドポイントの状態を確認します。

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

### Deploy model

<h5>最新モデルバージョンの取得</h5>
デプロイする登録済みモデルの最新バージョンを取得します。

In [ ]:
# Let's pick the latest version of the model
registered_model_name = "diabetes_model_01"

latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)
print("latest model version is :", latest_model_version)

<h5> オンラインデプロイメントの作成</h5>
取得したモデルを使用してオンライン推論用のデプロイメントを作成します。

In [ ]:
# picking the model to deploy. Here we use the latest version of our registered model
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)


# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment)

<h5> Blue デプロイメントへのルーティング割合の設定</h5>
Blue デプロイメントへのルーティングを 100% に変更します。

In [ ]:
# Traffic routing: 100% to blue deployment
# endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

print("endpoint:", online_endpoint_name)

<h5>テスト用データの準備</h5>
オンライン推論をテストするためのサンプルデータをJSONファイルで作成します。

In [ ]:
%%writefile ./sample-request.json
{"input_data": {
    "columns": ["age", "gender", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
    "data":[
      [0.038075906,0.050680119,0.061696207,0.021872386,-0.044223498,-0.034820763,-0.043400846,-0.002592262,0.019907486,-0.017646125]
    ],
    "index": [1]    
  }}

<h5>オンライン推論のテスト</h5>
作成したテストデータを使用してオンラインエンドポイントの推論をテストします。

In [ ]:
online_endpoint_name = "ep-online-oh4ml-01"

# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./sample-request.json",
    deployment_name="blue",
)

### Checking Credential

<h5>認証キーの取得</h5>
外部アプリケーションからエンドポイントにアクセスするための認証キーを取得します。

In [ ]:
endpoint_cred = ml_client.online_endpoints.get_keys(name=online_endpoint_name).primary_key
print(endpoint_cred)

### Deleting Online Endpoint

<h5> オンラインエンドポイントの削除</h5>
テスト完了後、コスト削減のためにオンラインエンドポイントを削除します。

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)